# Anti pauvreté

## Import packages and load data

In [119]:
import pandas as pd
import numpy as np
import folium
import geopy

In [12]:
df = pd.read_csv("hebergements.csv", sep = ";")

In [13]:
df

,type,nom,adresse,code_postal,ville,telephone,precisions,geo
0,CHU,Les Baudemons,"30, rue des Baudemons",94320.0,Thiais,01.55.53.16.10,Hommes et femmes isolés de plus de 50 ans.,"48.769465,2.394079"
1,CHRS,Poterne des Peupliers,"8-14, rue de la Poterne des Peupliers",75013.0,Paris,01.45.81.90.00,Hommes et femmes isolés de 25 à 65 ans. Couples.,"48.819423,2.353209"
2,CHU,Baudricourt,"15, rue Baudricourt",75013.0,Paris,01.45.83.32.60,Hommes et femmes isolés de 25 à 65 ans.,"48.828012,2.364797"
3,CHRS,Pauline Roland,"35-37, rue Fessart",75019.0,Paris,01.42.03.26.28,Femmes seules ou femmes avec enfants.,"48.876763,2.384943"
4,CHRS,Relais des carrières,"71, rue du Château des Rentiers",75013.0,Paris,01.42.16.76.76,Hommes et femmes isolés de 25 à 65 ans.,"48.826222,2.369066"
5,CHU,Crimée,"166, rue de Crimée",75019.0,Paris,01.53.26.53.26,Familles.,"48.889674,2.378039"
6,CHRS-CHU,Stendhal,"5 quater, rue Stendhal",75020.0,Paris,01.46.62.57.57,Hommes et femmes de 18 à 27 ans. Couples.,"48.861052,2.402218"
7,CHRS,Charonne,"43-45, boulevard de Charonne",75011.0,Paris,01.55.25.57.57,Familles monoparentales.,"48.852109,2.397565"


## Afficher une carte avec les centres d'hébergements d'Ile-de-France

In [22]:
df.loc[:, "Lat"] = df.geo.apply(lambda x : x.split(",")[0])
df.loc[:, "Long"] = df.geo.apply(lambda x : x.split(",")[1])

In [135]:
carte = folium.Map(location = [48.856614, 2.3522219], zoom_start = 12)

for i in range(len(df)) : 
    if df.loc[i, "type"] == 'CHU' : 
        color = "red"
    else :
        color = "green"
    folium.Marker([df.loc[i,"Lat"], df.loc[i,"Long"]], popup = df.loc[i, "type"] + " " + df.loc[i, "nom"], 
                 icon=folium.Icon(color=color, icon="info-sign")).add_to(carte)

In [136]:
carte

## Prendre une coordonnée GPS et trouver un itinéraire jusqu'à un centre d'hébergement

In [132]:
my_coo = [48.8865792, 2.2478848] # coordonnée GPS de l'utilisateur récupérée par l'application
folium.Marker(my_coo, popup = "Je suis ici").add_to(carte)

In [126]:
# fonction pour calculer le plus court chemin entre la position de l'utilisateur et tous les centres
def calcul_plus_court_chemin(coord, list_coord_test) : 
    list_distance = []
    for coord_test in list_coord_test : 
        list_distance.append(geopy.distance.distance(coord, coord_test).km)
    return np.argmin(list_distance), np.min(list_distance)

In [133]:
centre, distance_km = calcul_plus_court_chemin(my_coo, list(df["geo"])) # on prend le centre le plus proche

route = folium.PolyLine([my_coo, [df.loc[centre,"Lat"], df.loc[centre,"Long"]]], 
                       tooltip = "Itinéraire jusqu'au centre d'hébergement le plus proche - " + str(int(distance_km))
                        + " km", color="red", weight=4, opacity=0.6).add_to(carte)
carte